In [ ]:
#helper functions
mse_loss <- function(predicted, actual) {
  return(mean((predicted - actual) ^ 2))
}


In [ ]:
# some training data 
set.seed(64145894)

N_train = 250
f_true = function(x)x/3+x^5
x = runif(N_train)
eps = rnorm(N_train,0,1/25)
y = f_true(x) + eps

In [ ]:
plot(x,y)

In [ ]:
# sigmoid
activation <- function(x) {
  return(1 / (1 + exp(-x)))
}

# RELU
activation <- function(x) {
  return(map_vec(x,~max(0,.x)))
}

forward_propagation <- function(x, W1, b1, W2, b2) {
  x <- array(x,c(1,1))
  h1 = activation(W1 %*% x + b1)
  output = W2 %*% h1 + b2
  return(as.vector(output))
}


In [ ]:
# init
n_input <- 1
n_output <- 1
n_hidden <- 5
W1 <- abs(array(rnorm(n_input * n_hidden), c(n_hidden, n_input)))
b1 <- abs(rnorm(n_hidden))
W2 <- abs(array(rnorm(n_output * n_hidden), c(n_output, n_hidden)))
b2 <- abs(rnorm(n_output))


In [ ]:
library('purrr')
y_hat <- map_vec(x,~forward_propagation(.x,W1, b1, W2, b2))
plot(x,y_hat)

## Now we learn the parameters

In [ ]:
library('nloptr')

In [ ]:
to_theta = function(W1,b1,W2,b2){
    c(as.vector(W1),b1,as.vector(W2),b2)
}

to_mtxes = function(theta){
    W1 <- array(theta[1:n_hidden], c(n_hidden, n_input))
    b1 <- theta[(n_hidden+1):(2*n_hidden)]
    W2 <- array(theta[(2*n_hidden+1):(length(theta)-1)], c(n_output, n_hidden))
    b2 <- theta[length(theta)]
    return(list(W1=W1,b1=b1,W2=W2,b2=b2))
}

In [ ]:
W1

In [ ]:
theta = to_theta(W1,b1,W2,b2)
theta

In [ ]:
to_mtxes(theta)

In [ ]:
loss = function(theta){
    wts = to_mtxes(theta)
    y_hat <- map_vec(x,~forward_propagation(.x,wts$W1, wts$b1, wts$W2, wts$b2))
    l = mse_loss(y_hat,y)
    return(l)
}

In [ ]:
loss(theta)

In [ ]:
# random init
x0 = rnorm(length(theta))
x0

In [ ]:
optimze = function(x0 = runif(length(theta),-.3,.3)){
    opts <- list(
        "algorithm" = "NLOPT_LN_COBYLA", # Algorithm
        "xtol_rel" = 1e-10, # Relative tolerance on optimization
        "maxeval" = 500 # Maximum number of function evaluations
    )
    
    result <- nloptr(
        x0 = x0,
        eval_f = loss,
        opts = opts
    )

    obj = result$objective

    return(list(obj = obj, soln=result$solution,x0=x0))
}

    


In [ ]:
set.seed(NULL)

In [ ]:
res = list()
N_opt = 10
for(i in 1:N_opt){
    res[[i]] <- optimze()
}

In [ ]:
i_sel <- which.min(map_vec(res,~.x$obj))

In [ ]:
result <- res[[i_sel]]

In [ ]:
wts0 = to_mtxes(x0)
y_hat0 <- map_vec(x,~forward_propagation(.x,wts0$W1, wts0$b1, wts0$W2, wts0$b2))
wts = to_mtxes(result$soln)
y_hat <- map_vec(x,~forward_propagation(.x,wts$W1, wts$b1, wts$W2, wts$b2))
plot(x,y,col='red')
points(x,y_hat0,col='blue')
points(x,y_hat,col='black')

## fitting in keras

In [ ]:
set.seed(NULL)

In [ ]:
library('keras')

In [ ]:
model <- keras_model_sequential() %>%
    layer_dense(units = 5, input_shape = c(1),activation = 'relu') %>%
    layer_dense(units = 1)


In [ ]:
model %>% compile(
  optimizer = 'adam',
  loss = 'mse',
  metrics = c('mse')
)

In [ ]:
model %>% summary()

In [ ]:
history <- model %>% fit(
  x, y,
  epochs = 1000,
  batch_size = 32,
  validation_split = 0.2,  # Use 20% of the training data for validation
  verbose = 1
)

In [ ]:
plot(history)

In [ ]:
y_hat <- predict(model,x)
plot(x,y,col='red')
points(x,y_hat,col='black')